## Object Relation Model

In [10]:
import os
from sqlmodel import create_engine, Field, Session, select, SQLModel, Relationship


#### Single entity.

In [2]:
class Customer(SQLModel, table=True):
    """Class for managing customer data."""

    Id: int | None = Field(primary_key=True)
    Name: str
    Lastname: str
    Age: int | None

In [3]:
# Create a (SQLAlchemy) engine
engine = create_engine("sqlite:///orm-test.db", echo=True)

In [ ]:
# Create tables based on the model metadata.
SQLModel.metadata.create_all(engine)

In [ ]:
# Create a customer object
customer = Customer(Name="Ingrid", Lastname="Jansen")
customer

In [ ]:
# Create a session to the database.
with Session(engine) as session:
    # Add the customer record and commit.
    session.add(customer)
    session.commit()

In [ ]:
sql_customers = []
with Session(engine) as session:
    # Create the select query.
    query = select(Customer)

    # Perform the query on the database.
    result = session.exec(query)

    # Fetch all results.
    sql_customers = result.fetchall()


In [ ]:
# Note that the results are Python / SQLModel objects!
sql_customers

In [9]:
# Delete the engine / close connection
engine.dispose(close=True)
os.remove("orm-test.db")

#### Relationships

In [ ]:
class Customer(SQLModel, table=True):
    """Class for managing customer data."""

    CustomerId: int | None = Field(primary_key=True)
    AddressId: int
    Name: str
    Lastname: str
    Age: int | None



In [ ]:
class Address(SQLModel, table=True):
    """Class for managing address data."""

    AddressId: int | None = Field(primary_key=True)
    Street: str
    HouseNumber: str
    PostCode: str